# Use-case
Allows for executing an operation on the datastructure without modifying classes of the elements it works with.

# Situation when usage is suggested
* Object structure consists of many classes of many interfaces, and depending on the concrete implementation different resolution is required.
* To avoid mess in class representing datastructure, when multiple operation would have to be defined and executed, or when given operations are shared between multiple datastructures.
* In case new operation on the datastructure are implemented frequently, to avoid redefining the interface of said structure.

# Structure
![title](img/visitor.jpg)

# Implementation

In [1]:
class Visitor:
    _methods = {}

    @staticmethod
    def _qualname(obj):
        """Get the fully-qualified name of an object (including module)."""
        return obj.__module__ + '.' + obj.__qualname__

    def _declaring_class(self, obj):
        """Get the name of the class that declared an object."""
        name = self._qualname(obj)
        return name[:name.rfind('.')]

    @staticmethod
    def _visitor_impl(instance, arg):
        """Actual visitor method implementation."""
        method = Visitor._methods[(Visitor._qualname(type(instance)), type(arg))]
        return method(instance, arg)

    def __call__(self, arg_type):
        """Decorator that creates a visitor method."""

        def decorator(fn):
            declaring_class = self._declaring_class(fn)
            type(self)._methods[(declaring_class, arg_type)] = fn
            return self._visitor_impl

        return decorator


visitor = Visitor()

In [19]:
class Expression:
    def accept(self, visitor):
        visitor.visit(self)


class NumericExpression(Expression):
    def __init__(self, value):
        self.value = value


class AdditionExpression(Expression):
    def __init__(self, left, right):
        self.left = left
        self.right = right


# REMOVE
class MultiplicationExpression(Expression):
    def __init__(self, left, right):
        self.left = left
        self.right = right


expression_1 = AdditionExpression(
    NumericExpression(1),
    AdditionExpression(
        NumericExpression(2),
        NumericExpression(3)
    )
)

expression_2 = MultiplicationExpression(
    AdditionExpression(
        NumericExpression(1),
        NumericExpression(1)
    ),
    AdditionExpression(
        NumericExpression(2),
        NumericExpression(2)
    )
)

In [20]:
class ExpressionPrinter:
    def __init__(self):
        self.buffer = []

    @visitor(NumericExpression)
    def visit(self, ne):
        self.buffer.append(str(ne.value))

    @visitor(AdditionExpression)
    def visit(self, ae):
        self.buffer.append('(')
        ae.left.accept(self)
        self.buffer.append('+')
        ae.right.accept(self)
        self.buffer.append(')')

    # REMOVE
    @visitor(MultiplicationExpression)
    def visit(self, me):
        self.buffer.append('(')
        me.left.accept(self)
        self.buffer.append('*')
        me.right.accept(self)
        self.buffer.append(')')

    def __str__(self):
        return ''.join(self.buffer)


printer = ExpressionPrinter()
printer.visit(expression_1)
print(printer)

printer = ExpressionPrinter()
printer.visit(expression_2)
print(printer)

(1+(2+3))
((1+1)*(2+2))


In [21]:
# REMOVE
class ExpressionSolver:
    def __init__(self):
        self.value = None

    @visitor(NumericExpression)
    def visit(self, ne):
        self.value = ne.value

    @visitor(AdditionExpression)
    def visit(self, ae):
        ae.left.accept(self)
        temp = self.value
        ae.right.accept(self)
        self.value += temp

    @visitor(MultiplicationExpression)
    def visit(self, me):
        me.left.accept(self)
        temp = self.value
        me.right.accept(self)
        self.value *= temp

    def __str__(self):
        return f"{self.value}"

printer = ExpressionSolver()
printer.visit(expression_1)
print(printer)

printer = ExpressionSolver()
printer.visit(expression_2)
print(printer)

6
8


# Consequences
* __Simplifies implementation of new operations__ - Implementing new operation for datastructure is done by simply adding new visitor, on the contrary to modifying every single class to od so without the given design pattern.

* __Bundling of operations__ - In case of multiple operation required to perform a given task (visit) all of them will be placed inside one of implementations of Visitor class.
* __Allows visiting even unrelated objects__ - Objects do not have to inherit from the same parent to apply visitor pattern.
* __Complicates adding new datastructures__ - When datastructure inherits an interface adding a new datastructure forces implementation of method to handle that class in the visitor.
* __May violate encapsulation__ - While using visitor pattern, it may be necessary to allow the visitor to use methods that modify internal state of the object, which may violate encapsulation.
